Environment: This script should be run with the `python_plant_pathogen_atlas` environment using the devcontainer `docker_python`

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os, sys
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

In [ ]:
data_folder = r'../../data'
scrna = os.path.join(data_folder, 'temp_objects', 'seq_reference_00.h5ad')

Read in the RNA counts in the snMultiome object

In [ ]:
adata = sc.read(scrna)

adata.obs = adata.obs[[i for i in adata.obs.columns if 'pANN' not in i]]
adata.obs = adata.obs[[i for i in adata.obs.columns if 'DF.classifications' not in i]]

sc.pl.umap(adata, color=['SCT_snn_res.1', 'BON3', 'celltype'], vmax=1, legend_loc= 'on data')

Subset to only mesophyll cells and preprocess the RNA counts

In [ ]:
adata = adata[(adata.obs['celltype'].isin(['Mesophyll']))]
adata.obs_names_make_unique()

adata.raw=adata

sc.pp.recipe_zheng17(adata)

sc.tl.pca(adata, random_state=0)
sc.pp.neighbors(adata, random_state=0)
sc.tl.umap(adata, random_state=0)

For each mock cell, set it as the starting cell of a pseudotime trajectory

In [ ]:
mock_indices = np.where(adata.obs['sample'].isin(['00_Mock_rep1', '00_Mock_rep2']))[0]
pseudotimes =[]
for idx in mock_indices:
    adata.uns['iroot'] = idx
    sc.tl.dpt(adata, n_dcs=2)
    pseudotime = adata.obs['dpt_pseudotime']
    pseudotimes.append(pseudotime)

The total pseudotime trajectory is the mean of all of them

In [ ]:
adata.obs['dpt_pseudotime'] = np.mean(np.vstack([i.values for i in pseudotimes]), axis = 0)

sc.pl.umap(adata, color='dpt_pseudotime')

Add the pseudotime of the mesophyll cells to the original scRNA object

In [ ]:
scrna_copy = sc.read(scrna)

scrna_copy.obs = scrna_copy.obs[[i for i in scrna_copy.obs.columns if 'pANN' not in i]]
scrna_copy.obs = scrna_copy.obs[[i for i in scrna_copy.obs.columns if 'DF.classifications' not in i]]

scrna_copy.obs = scrna_copy.obs.merge(adata.obs.dpt_pseudotime, left_index=True, right_index=True, how='left')

scrna_copy.write(os.path.join(data_folder, 'temp_objects', 'seq_pseudotime_01.h5ad'))